In [1]:
import json
import os
import sys
from copy import deepcopy
from pathlib import Path

import pandas as pd

REPO_DIR = os.path.abspath('..')  # path to the root of the repository
sys.path.append(REPO_DIR)
os.environ["PROJECT_DIR"] = REPO_DIR
import lib

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
CALIFORNIA = 'california_housing'
ADULT = 'adult'
HELENA = 'helena'
JANNIS = 'jannis'
HIGGS = 'higgs_small'
ALOI = 'aloi'
EPSILON = 'epsilon'
YEAR = 'year'
COVTYPE = 'covtype'
YAHOO = 'yahoo'
MICROSOFT = 'microsoft'
ALL_DATASETS = [CALIFORNIA, ADULT, HELENA, JANNIS, HIGGS, ALOI, EPSILON, YEAR, COVTYPE, YAHOO, MICROSOFT]
DATASET_NAMES = {
    CALIFORNIA: 'California Housing',
    ADULT: 'Adult',
    HELENA: 'Helena',
    JANNIS: 'Jannis',
    HIGGS: 'Higgs Small',
    ALOI: 'ALOI',
    EPSILON: 'Epsilon',
    YEAR: 'Year',
    COVTYPE: 'Covertype',
    YAHOO: 'Yahoo',
    MICROSOFT: 'Microsoft',
}
REGRESSION_DATASETS = [x for x in ALL_DATASETS if lib.load_dataset_info(x)['task_type'] == lib.REGRESSION]
DETAILS = ['task_type', 'n_objects', 'n_features']
PARTS = ['test', 'val', 'train']


def format_scores(df, precision):
    def f(record):
        if record['task_type'] == lib.REGRESSION:
            for part in PARTS:
                for suffix in 'best', 'score':
                    key = f'{part}_{suffix}'
                    if key in record:
                        record[key] *= -1
        for k, v in list(record.items()):
            if isinstance(v, float):
                record[k] = round(v, precision)
        return record
    return df.apply(f, axis=1)


def load_record(output):
    output = Path(output)
    if not output.exists():
        return None
    path = output / 'stats.json'
    if not path.exists():
        print(f'WARNING! This path does not exist: {path}')
        return None
    stats = lib.load_json(path)
    metrics = stats.get('metrics')
    if metrics is None:
        return None

    dataset = Path(stats['dataset']).name
    info = lib.load_dataset_info(dataset)
    dataset, algorithm, experiment, suffix = str(output.relative_to(lib.env.OUTPUT_DIR)).split('/', 4)
    r = {
        'dataset': DATASET_NAMES[dataset],
        'task_type': info['task_type'],
        'n_objects': info['size'],
        'n_features': info['n_num_features'] + info['n_cat_features'],
        'algorithm': algorithm + f' | {experiment}',
        's': suffix
    }
    for x in PARTS:
        if x in stats['metrics']:
            r[f'{x}_score'] = stats['metrics'][x]['score']
    return r


def sort(df, by):
    if isinstance(by, str):
        by = [by]
    return df.sort_values(['n_objects'] + by, ascending=[True] + ['score' not in x for x in by]).reset_index(drop=True)


def make_df(outputs_and_names):
    df = []
    for output, algorithm_name in outputs_and_names:
        record = load_record(output)
        if not record:
            continue
        if algorithm_name is not None:
            record['algorithm'] = algorithm_name
        df.append(record)
    df = sort(pd.DataFrame(df).fillna(0.0), 'val_score').reset_index(drop=True)
    return df


def collect_outputs(experiment_dir, filter_info=None):
    if isinstance(filter_info, int):
        filter_info = [str(x) for x in range(filter_info)]
    if isinstance(filter_info, list):
        assert all(isinstance(x, str) for x in filter_info)
        filter_fn = lambda x: x.name in filter_info
    elif callable(filter_info):
        filter_fn = filter_info
    else:
        assert filter_info is None
        filter_fn = lambda x: True

    outputs = []
    if not isinstance(experiment_dir, Path):
        experiment_dir = lib.env.OUTPUT_DIR / experiment_dir
    if experiment_dir.exists():
        outputs.extend(
            filter(
                filter_fn,
                filter(Path.is_dir, experiment_dir.iterdir())
            )
        )
    return outputs


def aggregate(df):
    aggrs = dict(
        task_type=('task_type', 'first'),
        n_objects=('n_objects', 'first'),
        n_features=('n_features', 'first'),
        test_score=('test_score', 'mean'),
        test_std=('test_score', 'std'),
        val_score=('val_score', 'mean'),
        val_std=('val_score', 'std'),
        count=('test_score', 'count')
    )
    if 'train_score' in df.columns:
        aggrs.update(dict(
            train_score=('train_score', 'mean'),
            train_std=('train_score', 'std'),
        ))
    df = df.groupby(['dataset', 'algorithm']).agg(**aggrs)
    df['count'] = df['count'].astype(int)
    return df.reset_index().fillna(0.0)


def build_report(outputs_and_names):
    df = make_df(outputs_and_names)
    df = aggregate(df)
    df = sort(df, 'test_score')
    df = format_scores(df, 4)
    # df = df.set_index(['dataset', 'algorithm']).drop(columns=DETAILS)
    df = df.set_index(['dataset'] + DETAILS + ['algorithm'])
    return df[['test_score', 'test_std', 'val_score', 'val_std', 'train_score', 'train_std', 'count']]

## Default configurations (GBDT and FT-Transformer)

In [3]:
all_datasets = set(deepcopy(ALL_DATASETS))
n_seeds = 15
ensemble_names = ['0_4', '5_9', '10_14']
outputs_and_names = []
for experiment, algorithm_name, datasets in [
    ('ft_transformer/default', 'FT-Transformer', all_datasets),
    ('catboost/default', 'CatBoost', all_datasets),
    ('xgboost/default', 'XGBoost', all_datasets),
]:
    for dataset in datasets:
        for output in collect_outputs(dataset + '/' + experiment, n_seeds):
            outputs_and_names.append((output, algorithm_name))
        for output in collect_outputs(dataset + '/' + experiment + '_ensemble', ensemble_names):
            outputs_and_names.append((output, '(e) ' + algorithm_name))
build_report(outputs_and_names)

test_score  \
dataset            task_type  n_objects n_features algorithm                        
California Housing regression 20640     8          (e) CatBoost            0.4281   
                                                   CatBoost                0.4303   
                                                   (e) FT-Transformer      0.4543   
                                                   XGBoost                 0.4622   
                                                   (e) XGBoost             0.4622   
                                                   FT-Transformer          0.4688   
Adult              binclass   48842     14         XGBoost                 0.8741   
                                                   (e) XGBoost             0.8741   
                                                   (e) CatBoost            0.8735   
                                                   CatBoost                0.8727   
                                                   (e) FT-Transformer      0.8598   
                                                   FT-Transformer          0.8574   
Helena             multiclass 65196     27         (e) FT-Transformer      0.3948   
                                                   (e) CatBoost            0.3863   
                                                   CatBoost                0.3814   
                                                   FT-Transformer          0.3812   
                                                   (e) XGBoost             0.3479   
                                                   XGBoost                 0.3479   
Jannis             multiclass 83733     54         (e) FT-Transformer      0.7341   
                                                   FT-Transformer          0.7252   
                                                   (e) CatBoost            0.7243   
                                                   CatBoost                0.7210   
                                                   (e) XGBoost             0.7110   
                                                   XGBoost                 0.7110   
Higgs Small        binclass   98050     28         (e) FT-Transformer      0.7314   
                                                   (e) CatBoost            0.7278   
                                                   CatBoost                0.7255   
                                                   FT-Transformer          0.7250   
                                                   XGBoost                 0.7165   
                                                   (e) XGBoost             0.7165   
ALOI               multiclass 108000    128        (e) FT-Transformer      0.9665   
                                                   FT-Transformer          0.9530   
                                                   (e) CatBoost            0.9485   
                                                   CatBoost                0.9462   
                                                   XGBoost                 0.9242   
                                                   (e) XGBoost             0.9242   
Epsilon            binclass   500000    2000       (e) FT-Transformer      0.8969   
                                                   FT-Transformer          0.8959   
                                                   (e) CatBoost            0.8893   
                                                   CatBoost                0.8880   
                                                   XGBoost                 0.8799   
                                                   (e) XGBoost             0.8799   
Year               regression 515345    90         (e) FT-Transformer      8.7267   
                                                   (e) CatBoost            8.8845   
                                                   FT-Transformer          8.8892   
                                                   CatBoost                8

## All Neural Networks

In [4]:
all_datasets = set(deepcopy(ALL_DATASETS))
n_seeds = 15
outputs_and_names = []
for experiment, algorithm_name, datasets in [
    # !!! TUTORIAL !!! Uncomment the following line for completing the tutorial:
    # ('mlp/tuned_reproduced', 'MLP | reproduced', [CALIFORNIA]),
    ('mlp/tuned', 'MLP', all_datasets),
    ('resnet/tuned', 'ResNet', all_datasets),
    ('snn/tuned', 'SNN', all_datasets),
    ('dcn2/tuned', 'DCN V2', all_datasets),
    ('tabnet/tuned', 'TabNet', all_datasets),
    ('grownet/tuned', 'GrowNet', all_datasets - {HELENA, JANNIS, ALOI, COVTYPE}),  # GrowNet does not support multiclass problems
    ('node/tuned', 'NODE', all_datasets - {HELENA, ALOI}),
    ('node/default', 'NODE', {HELENA, ALOI}),
    ('autoint/tuned', 'AutoInt', all_datasets),
    ('ft_transformer/tuned', 'FT-Transformer', all_datasets - {YAHOO}),
    ('ft_transformer/default', 'FT-Transformer | default', {YAHOO}),
]:
    for dataset in datasets:
        for output in collect_outputs(dataset + '/' + experiment, n_seeds):
            outputs_and_names.append((output, algorithm_name))
build_report(outputs_and_names)

test_score  \
dataset            task_type  n_objects n_features algorithm                              
California Housing regression 20640     8          FT-Transformer                0.4591   
                                                   NODE                          0.4636   
                                                   AutoInt                       0.4739   
                                                   DCN V2                        0.4837   
                                                   ResNet                        0.4856   
                                                   GrowNet                       0.4869   
                                                   SNN                           0.4933   
                                                   MLP                           0.4985   
                                                   TabNet                        0.5100   
Adult              binclass   48842     14         AutoInt                       0.8589   
                                                   FT-Transformer                0.8587   
                                                   NODE                          0.8577   
                                                   GrowNet                       0.8574   
                                                   SNN                           0.8544   
                                                   ResNet                        0.8544   
                                                   DCN V2                        0.8529   
                                                   MLP                           0.8522   
                                                   TabNet                        0.8502   
Helena             multiclass 65196     27         ResNet                        0.3960   
                                                   FT-Transformer                0.3913   
                                                   DCN V2                        0.3853   
                                                   MLP                           0.3832   
                                                   TabNet                        0.3782   
                                                   SNN                           0.3728   
                                                   AutoInt                       0.3722   
                                                   NODE                          0.3593   
Jannis             multiclass 83733     54         FT-Transformer                0.7323   
                                                   ResNet                        0.7280   
                                                   NODE                          0.7271   
                                                   TabNet                        0.7230   
                                                   AutoInt                       0.7211   
                                                   MLP                           0.7195   
                                                   SNN                           0.7192   
                                                   DCN V2                        0.7159   
Higgs Small        binclass   98050     28         FT-Transformer                0.7290   
                                                   ResNet                        0.7266   
                                                   NODE                          0.7263   
                                                   AutoInt                       0.7254   
                                                   DCN V2                        0.7230   
                                                   MLP                           0.7226   
                                                   SNN                           0.7221   
                                                   GrowNet                       0.7216   
                                                   TabNet                

## Main Neural Networks and GDBT

In [5]:
all_datasets = set(deepcopy(ALL_DATASETS))
n_seeds = 15
ensemble_names = ['0_4', '5_9', '10_14']
outputs_and_names = []
for experiment, algorithm_name, datasets in [
    ('mlp/tuned', 'MLP', all_datasets),
    ('resnet/tuned', 'ResNet', all_datasets),
    ('ft_transformer/tuned', 'FT-Transformer', all_datasets - {YAHOO}),
    ('ft_transformer/default', 'FT-Transformer | default', all_datasets),
    ('catboost/tuned', 'CatBoost', all_datasets),
    ('xgboost/tuned', 'XGBoost', all_datasets),
    ('lightgbm_/tuned', 'LightGBM', {CALIFORNIA, ADULT, HIGGS}),
]:
    for dataset in datasets:
        for output in collect_outputs(dataset + '/' + experiment, n_seeds):
            outputs_and_names.append((output, algorithm_name))
        for output in collect_outputs(dataset + '/' + experiment + '_ensemble', ensemble_names):
            outputs_and_names.append((output, '(e) ' + algorithm_name))
build_report(outputs_and_names)

test_score  \
dataset            task_type  n_objects n_features algorithm                                  
California Housing regression 20640     8          (e) CatBoost                      0.4234   
                                                   (e) LightGBM                      0.4268   
                                                   CatBoost                          0.4308   
                                                   (e) XGBoost                       0.4314   
                                                   LightGBM                          0.4322   
                                                   XGBoost                           0.4334   
                                                   (e) FT-Transformer                0.4480   
                                                   (e) FT-Transformer | default      0.4543   
                                                   FT-Transformer                    0.4591   
                                                   FT-Transformer | default          0.4688   
                                                   (e) ResNet                        0.4780   
                                                   ResNet                            0.4856   
                                                   (e) MLP                           0.4869   
                                                   MLP                               0.4985   
Adult              binclass   48842     14         (e) CatBoost                      0.8742   
                                                   CatBoost                          0.8727   
                                                   (e) LightGBM                      0.8723   
                                                   LightGBM                          0.8723   
                                                   XGBoost                           0.8723   
                                                   (e) XGBoost                       0.8722   
                                                   (e) FT-Transformer                0.8601   
                                                   (e) FT-Transformer | default      0.8598   
                                                   FT-Transformer                    0.8587   
                                                   FT-Transformer | default          0.8574   
                                                   (e) ResNet                        0.8571   
                                                   (e) MLP                           0.8548   
                                                   ResNet                            0.8544   
                                                   MLP                               0.8522   
Helena             multiclass 65196     27         (e) FT-Transformer                0.3983   
                                                   (e) ResNet                        0.3981   
                                                   ResNet                            0.3960   
                                                   (e) FT-Transformer | default      0.3948   
                                                   FT-Transformer                    0.3913   
                                                   (e) MLP                           0.3902   
                                                   (e) CatBoost                      0.3885   
                                                   CatBoost                          0.3853   
                                                   MLP                               0.3832   
                                                   FT-Transformer | default          0.3812   
                                                   (e) XGBoost                       0.3775   
                                                   XGBoost                           0.3754   
Jannis             multiclass 83733     54         (e) FT-Transformer                0.7390

## Ablation Study

In [6]:
all_datasets = {CALIFORNIA, HELENA, JANNIS, HIGGS, ALOI, YEAR, COVTYPE, MICROSOFT}
n_seeds = 15
outputs_and_names = []
for experiment, algorithm_name, datasets in [
    ('autoint/tuned', 'AutoInt', all_datasets),
    ('ft_transformer/tuned_nobias', 'FT-Transformer | nobias', all_datasets),
    ('ft_transformer/tuned', 'FT-Transformer', all_datasets),
]:
    for dataset in datasets:
        for output in collect_outputs(dataset + '/' + experiment, n_seeds):
            outputs_and_names.append((output, algorithm_name))
build_report(outputs_and_names)

test_score  \
dataset            task_type  n_objects n_features algorithm                             
California Housing regression 20640     8          FT-Transformer               0.4591   
                                                   FT-Transformer | nobias      0.4701   
                                                   AutoInt                      0.4739   
Helena             multiclass 65196     27         FT-Transformer               0.3913   
                                                   FT-Transformer | nobias      0.3811   
                                                   AutoInt                      0.3722   
Jannis             multiclass 83733     54         FT-Transformer               0.7323   
                                                   FT-Transformer | nobias      0.7243   
                                                   AutoInt                      0.7211   
Higgs Small        binclass   98050     28         FT-Transformer               0.7290   
                                                   FT-Transformer | nobias      0.7271   
                                                   AutoInt                      0.7254   
ALOI               multiclass 108000    128        FT-Transformer               0.9605   
                                                   FT-Transformer | nobias      0.9579   
                                                   AutoInt                      0.9452   
Year               regression 515345    90         FT-Transformer | nobias      8.8429   
                                                   FT-Transformer               8.8550   
                                                   AutoInt                      8.8817   
Covertype          multiclass 581012    54         FT-Transformer               0.9697   
                                                   FT-Transformer | nobias      0.9642   
                                                   AutoInt                      0.9337   
Microsoft          regression 1200192   136        FT-Transformer               0.7462   
                                                   AutoInt                      0.7501   
                                                   FT-Transformer | nobias      0.7513   

                                                                            test_std  \
dataset            task_type  n_objects n_features algorithm                           
California Housing regression 20640     8          FT-Transformer             0.0036   
                                                   FT-Transformer | nobias    0.0060   
                                                   AutoInt                    0.0034   
Helena             multiclass 65196     27         FT-Transformer             0.0013   
                                                   FT-Transformer | nobias    0.0017   
                                                   AutoInt                    0.0027   
Jannis             multiclass 83733     54         FT-Transformer             0.0021   
                                                   FT-Transformer | nobias    0.0041   
                                                   AutoInt                    0.0024   
Higgs Small        binclass   98050     28         FT-Transformer             0.0016   
                                                   FT-Transformer | nobias    0.0020   
                                                   AutoInt                    0.0018   
ALOI               multiclass 108000    128        FT-Transformer             0.0011   
                                                   FT-Transformer | nobias    0.0013   
                                                   AutoInt                    0.0014   
Year               regression 515345    90         FT-Transformer | nobias    0.0265   
                                                   FT-Transformer             0.0325   
                                                   AutoInt     